# Assignment 1

## 목표: 한국어 신문기사를 가지고 n-gram에 기반한 Language Modeling 구현
- File Handling
- Preprocessing : kss 모듈 등, preprosessing 좀 더 해보기!
- Word Count
- Efficient한 N-gram 구현


## 내용:

- NLRW1900000011.json 은 강원일보의 뉴스기사이다. 이 파일만을 가지고 언어 모델링 할 것
- 파일에서 신문기사만을 추출하고, 각 기사를 문장단위로 분리한 후 문장 처음과 끝에 각각 < s, /s >표시를 붙일 것
- 뉴스기사를 가급적 정리할 것. Regular expression, 한국어 형태소 분석기 사용, translation 등으로 필요없는 글자들을 제거
- 정리된 데이터를 가지고 실제 기사에 나오는 **'21세기는 선진 경제와 예술인의 역량을 높이는 일에 앞장서고자 한다'** 문장의 unigram, Bigram, Trigram 언어모델에 의한 확률을 각각 구하라
- **나타나지 않는 N-Gram 구성은 add-1 smoothing을 하라**
- 모듈화한 프로그램의 자세한 설명이 필요
- 실행결과가 반영된 상태로 주피터 노트북 탑재
- 화일 이름은 'Assignment1_학번_이름'.iypnb로 할 것
- **마감: 9월 27일 수요일 23시 59분 59초까지**
- 로그확률로 하고 검증 위해 로그 없는 걸로도1

## Data Loading and Preprocessing

#필요에 따라 셀을 더 자세히 나누어 구현 가능. 자세한 설명도 첨부할 필요

## Unigram Language Model에 의한 확률
### Unigram Model (k=1): $P(w_1 w_2 ... w_n) \approx \prod_{i} P(w_i)$

\begin{equation}
    P(x_1) \approx \frac{count(x_1)}{count(N)}
\end{equation}

- P(< s 이제 경기 침체로 여외 없이 영세 상인들이 피해를 입을 것으로 보인다 /s >)


## Bigram Language Model에 의한 확률

1.   항목 추가
2.   항목 추가


### Bigram Model (k=2): $P(w_i|w_1 w_2 ... w_{i-1}) \approx P(w_i|w_{i-1})$

\begin{equation}
    P(x_2|x_1) \approx \frac{count(x_1,x_2)}{count(x_1)}
\end{equation}

- P(< s 이제 경기 침체로 여외 없이 영세 상인들이 피해를 입을 것으로 보인다 /s >)


## Trigram Language Model에 의한 확률
\begin{equation}
    P(x_3|x_1,x_2) \approx \frac{count(x_1,x_2,x_3)}{count(x_1,x_2)}
\end{equation}

- P(< s 이제 경기 침체로 여외 없이 영세 상인들이 피해를 입을 것으로 보인다/s >)

# 0. 필요한 library 다운로드 및 파일 경로 설정
1. sentence tokenize를 위해 kss와 한국어 word tokenize를 위한 mecab,konlpy을 설치한다.
* kss version: 2.5.0
* konlpy version: 0.6.0
* MeCab version: 0.996


2. 본 ipynb 파일이 있는 경로와 같은 경로에 NLRW1900000011.json 파일을 넣는다.

In [1]:
!pip install kss==2.5.0
!pip install konlpy
!pip install mecab-python3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 8.1 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,283 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,260 kB]
Get:13 http://archive.ubuntu.com/ubuntu jamm

In [3]:
import konlpy
import MeCab
print("konlpy version:", konlpy.__version__)
print("MeCab version:", MeCab.VERSION)
!mecab-config --version

konlpy version: 0.6.0
MeCab version: 0.996
0.996/ko-0.9.2


# 1. json 파일 불러오기

In [4]:
import sys
import json
import re
import kss

with open('NLRW1900000011.json') as f:
    data = json.load(f)

# 2. Sentence Tokenize
가져온 json파일에서 sentence들만 가져와서 sentences라는 list에 저장한 뒤 이를 return하는 함수인 sentence_tokenize를 만들었다.

*   processing 과정이 얼마나 진행되었는지 확인시키기 위해, 10000의 배수마다 한 번씩 진행상황을 출력한다.
*   kss 모듈을 이용해 sentence를 tokenize한다.



In [5]:
def senetence_tokenize(data):
    paragraphs = []
    sentences = []
    i=0
    for document in data["document"]:
      for paragraph in document["paragraph"]:
        paragraphs.append(paragraph["form"])
    paragraph_length=len(paragraphs)
    for paragraph in paragraphs:
      paragraph_splited = kss.split_sentences(paragraph)
      if i %10000 == 0:
        print("Processing ",i,"/",paragraph_length,"th paragraph")
      i+=1
      # if i > 1000:
      #   break
      for sentence in paragraph_splited:
          if sentence != '':
              sentences.append(sentence)
    return sentences

In [6]:
sentences=senetence_tokenize(data)

Processing  0 / 69731 th paragraph
Processing  10000 / 69731 th paragraph
Processing  20000 / 69731 th paragraph
Processing  30000 / 69731 th paragraph
Processing  40000 / 69731 th paragraph
Processing  50000 / 69731 th paragraph
Processing  60000 / 69731 th paragraph


In [7]:
print(sentences[0:20])
print(len(sentences))

['새로운 희망 공유하고 새 출발하자', '기축년(己丑年) 새해다.', '새로운 희망을 공유하고 새 출발을 다짐할 때다.', '짙푸른 동해바다를 뚫고 이글거리는 광채를 뿜으며 힘차게 솟아오르는 태양을 안는다.', '어제와 다른 이 아침의 햇살을 받으며 왜 간절한 소망이 없겠는가.', '지난날의 갈등과 혼란 회한을 뒤로하고 새로 일어서야 한다.', '올해는 불신과 증오, 암투와 음모, 좌절과 절망 등 어둡고 참담한 것들이 걷히기를 소망한다.', '경제위기라고 하지만 우리는 저력을 가지고 있다.', '남북관계 새 틀 모색해야', '우리의 역사는 위기와 극복의 과정으로 이어져 왔다.', '시련과 고난을 극복하는 과정에서 오히려 민족의 역량을 키우고 한 단계 발전을 이룩해 왔던 것이다.', '달군 쇠가 단단해지고, 비 온 뒤의 땅이 굳어진다는 것은 마치 우리 민족을 두고 한 듯하다.', '이것은 우리의 무서운 생명력이다.', '그 피가 지금 우리 몸속에 흐르고 있다.', '‘대한민국호’는 글로벌 경제위기라는 미증유의 시련을 떨쳐내고 선진국으로 도약해야 한다.', '국민과 정부가 한마음 한뜻으로 경제 살리기에 나선다면 재도약은 못 이룰 꿈도 아니다.', '이를 위해서는 첫 단추를 잘 끼워야 한다.', '정부가 해야 할 몫은 경제에 자생력과 추진력을 불러일으키는 것이다.', '기업과 국민이 의욕적으로 다시 뛸 수 있도록 동기를 부여하고 여건을 만들어 주는 일이다.', '이를 바탕으로 변화의 바람을 몰고 올 ‘오바마 코드’를 제대로 읽어야 한다.']
94899


# 3. Preprocessing

##3.1 Sentence Cleaning


*   clean 함수
  * mapping에 포함된 문자들을 해당 매핑 값으로 치환.
  * punct에 포함된 구두점들을 각각 앞뒤에 공백을 추가하여 치환.
  * 특정 특수문자들을 미리 정의된 값으로 치환.
  * 최종적으로 정제된 텍스트를 반환.

*   clean_str 함수
  * 한글 자모를 제거
  * 특수 기호를 제거
  * 특정 특수문자들을 제거하거나 치환
  * 줄바꿈 문자를 마침표로 치환
  * 최종적으로 정제된 텍스트를 반환



In [8]:
def clean(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    for p in punct:
        text = text.replace(p, f' {p} ')
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])
    return text.strip()


def clean_str(text):
    text = re.sub(pattern='([ㄱ-ㅎㅏ-ㅣ]+)', repl='', string=text)
    text = re.sub(pattern='[^\w\s\n]', repl='', string=text)
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', string=text)
    text = re.sub('\n', '.', string=text)
    return text

##3.2 Word Tokenize

word_tokenize라는 함수를 정의하고, sentence를 Input으로 받아 clean, clean_str 함수를 실행한다. 또, mecab을 통해 형태소 분석을 실행한다. 마지막으로 문장의 시작과 끝에 "<s","\s>"를 삽입한다.

In [9]:
def word_tokenize(sentence,m,punct,punct_mapping):
  sen=clean(sentence,punct,punct_mapping);
  sen=clean_str(sen)
  token = m.morphs(sen)
  token.insert(0, '<s')
  token.append('/s>')
  return token

## 3.3 token_list 생성

모든 sentence에 대해 word_tokenize를 실행하고, 그 결과를 token_list라는 list에 넣는다. 그 결과는 list 안의 list로 생성되기 때문에, itertools를 이용해, merged_list라는 1차원 리스트를 만들어 준다.

In [10]:
from konlpy.tag import Mecab
import itertools

m = Mecab()
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
token_list=[]
for sent in sentences:
  token_list.append(word_tokenize(sent,m,punct,punct_mapping))

merged_list=list(itertools.chain(*token_list))

In [11]:
print(merged_list[0:20])

['<s', '새로운', '희망', '공유', '하', '고', '새', '출발', '하', '자', '/s>', '<s', '기축', '년', '己', '丑年', '새', '해다', '/s>', '<s']


## 3.4 make_merged_list

편의를 위해 json_path를 받아서 merged_list를 생성하는 과정까지를 make_merged_list라는 wrapper function으로 만들었다.

In [12]:
def make_merged_list(json_path):
  with open(json_path) as f:
    data = json.load(f)
  sentences=senetence_tokenize(data)
  m = Mecab()
  punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
  punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
  token_list=[]
  for sent in sentences:
    token_list.append(word_tokenize(sent,m,punct,punct_mapping))

  merged_list=list(itertools.chain(*token_list))
  return merged_list

# 4. ngram 만들기

## 4.1 generate_ngrams
리스트 컴프리헨션을 사용하여, 입력 리스트의 각 항목을 기준으로 n개의 연속적인 항목들을 묶어서 리스트로 만든다.
zip(*...)을 사용하여 n개의 리스트를 병렬로 묶어 튜플의 리스트로 변환하고, 최종적으로 n-gram들이 튜플로 묶인 리스트를 반환한다. n값에 따라 unigram, bigram, trigram을 만들 수 있다.

In [13]:
def generate_ngrams(input_list, n):
    return list(zip(*[input_list[i:] for i in range(n)]))

## 4.2 generate_ngram_counter

Counter 클래스를 사용하여 각 n-gram의 빈도를 계산한다.
n-gram과 해당 빈도를 포함하는 딕셔너리 형태로 반환한다.

In [14]:
from collections import Counter
def generate_ngram_counter(merged_list,n):
  seq=generate_ngrams(merged_list, n)
  count=Counter(seq)
  #print(sorted(count.items(), key = lambda x: x[1], reverse = True))
  return count

In [15]:
unigram_count=generate_ngram_counter(merged_list,1)
bigram_count=generate_ngram_counter(merged_list,2)
trigram_count=generate_ngram_counter(merged_list,3)

In [16]:
print(unigram_count)
print(bigram_count)
print(trigram_count)

Counter({('<s',): 94899, ('/s>',): 94899, ('이',): 85426, ('을',): 75037, ('는',): 74730, ('다',): 73330, ('의',): 56663, ('에',): 53638, ('하',): 46350, ('고',): 44259, ('를',): 40146, ('은',): 39168, ('있',): 34350, ('가',): 31109, ('도',): 27706, ('으로',): 26900, ('한',): 26850, ('에서',): 20777, ('로',): 20651, ('들',): 20421, ('등',): 19804, ('과',): 18822, ('했',): 17315, ('것',): 15797, ('적',): 14954, ('해',): 12692, ('와',): 12420, ('인',): 11877, ('기',): 11735, ('일',): 11607, ('지역',): 11470, ('되',): 10855, ('지',): 10360, ('수',): 10245, ('만',): 10003, ('할',): 9721, ('년',): 9585, ('1',): 8807, ('게',): 8724, ('며',): 8298, ('원',): 7694, ('어',): 7669, ('한다',): 7185, ('2',): 6852, ('명',): 6836, ('면',): 6204, ('3',): 6144, ('사업',): 6137, ('된',): 5803, ('까지',): 5548, ('않',): 5528, ('월',): 5470, ('강원',): 5387, ('성',): 5284, ('춘천',): 5031, ('었',): 4964, ('없',): 4861, ('개',): 4804, ('내',): 4753, ('5',): 4746, ('지만',): 4682, ('화',): 4643, ('권',): 4641, ('4',): 4616, ('말',): 4614, ('대',): 4561, ('여',): 4480, ('시',)

# 5. target sentence preprocessing
* target sentence를 정의하고, word_tokenize 함수를 씌어 preprocessing한다.
* target sentence에 대해 ngram을 만든다.

In [17]:
target_sentence = "21세기는 선진 경제와 예술인의 역량을 높이는 일에 앞장서고자 한다"
target_token=word_tokenize(target_sentence,m,punct,punct_mapping)
print(target_token)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['<s', '21', '세기', '는', '선진', '경제', '와', '예술인', '의', '역량', '을', '높이', '는', '일', '에', '앞장서', '고자', '한다', '/s>']


In [18]:
target_unigram = generate_ngrams(target_token, 1)
target_bigram = generate_ngrams(target_token, 2)
target_trigram = generate_ngrams(target_token, 3)

# 6. ngram probability 구하기

##6.1 find
token들의 list와 ngram_count에서를 받아, 이가 전체 언어에서, 즉, n-gram 빈도수를 count한 ngram_count에 저장된 빈도수를 구한다.

In [19]:
def find(array,ngram_count):
  #print("tuple",tuple(array))
  return ngram_count.get(tuple(array),0)

## 6.2 estimate_n-gram_prob



*   estimate_unigram_prob
  * 확률을 계산할 token w_i의 확률을 구하기 위해서
\begin{equation}
    P(w_i) = \frac{count(w_i)}{전체 count의 합}
\end{equation}
함수를 그대로 정의한 것이다.
단, is_add_one이라는 매개변수를 이용해서, 만약 이를 사용한다면, \begin{equation}
    P(w_i) = \frac{count(w_i)+1}{전체 count의 합+vocabulary 개수}
\end{equation}
를 계산해주는 것을 넣었다.
  * 이 때, 각 count는 앞서 정의한 find함수를 이용한다. 이를 위해, token을 list에 담아, unigram_count와 함께 find의 매개변수로 전달한다.
*   estimate_bigram_prob
  * estimate_unigram_prob과 비슷하게
  \begin{equation}
    P(w_i|w_{i-1}) = \frac{count(w_{i-1},w_i)}{count(w_{i-1})}
\end{equation} 이를 구하는 과정을 그대로 구현했다. is_add_one 사용도 동일하다.
*   estimate_trigram_prob
 * trigram 식을 위와 동일한 방법으로 구현했다.



In [20]:
def estimate_unigram_prob(w_curr, unigram_count, is_add_one):
    numerator = find([w_curr], unigram_count)
    denominator = sum(unigram_count.values())
    if is_add_one:
        vocabulary_size = len(unigram_count)
        return (numerator + 1) / (denominator + vocabulary_size)
    else:
        return numerator / denominator

In [21]:
def estimate_bigram_prob(w_prev, w_curr, unigram_count, bigram_count, is_add_one):
    numerator = find([w_prev, w_curr], bigram_count)
    denominator = find([w_prev],unigram_count)
    if is_add_one:
        vocabulary_size = len(unigram_count)
        #print("is_add_one : ",is_add_one,"\n w_prev :",w_prev,"\n w_curr : ",w_curr,"\n numerator",(numerator + 1) , "\n deno",(denominator + vocabulary_size))
        return (numerator + 1) / (denominator + vocabulary_size)
    else:
        #print("\nis_add_one : ",is_add_one,"\nw_prev :",w_prev,"\nw_curr : ",w_curr,"\nnumerator",(numerator ) , "\ndeno",(denominator))
        return numerator / denominator

In [22]:
def estimate_trigram_prob(w_prev1,w_prev2, w_curr, unigram_count, bigram_count,trigram_count, is_add_one):
    numerator = find([w_prev1, w_prev2, w_curr], trigram_count)
    denominator = find([w_prev1,w_prev2],bigram_count)
    if is_add_one:
        vocabulary_size = len(bigram_count)
        #print("is_add_one : ",is_add_one,"\n w_prev :",w_prev,"\n w_curr : ",w_curr,"\n numerator",(numerator + 1) , "\n deno",(denominator + vocabulary_size))
        return (numerator + 1) / (denominator + vocabulary_size)
    else:
        #print("\nis_add_one : ",is_add_one,"\nw_prev :",w_prev,"\nw_curr : ",w_curr,"\nnumerator",(numerator ) , "\ndeno",(denominator))
        return numerator / denominator

In [23]:
estimate_bigram_prob("새로운","희망",unigram_count, bigram_count, 1)

0.0004157592555929519

# 6.3 calculate_n-gram_prob

target_sentence를 tokenize한 결과와 n-gram_count를 input으로 넣으면, 이 seq가 나타날 확률을 구하는 것이다. 즉, \begin{equation}
    P(w_1 w_2 ... w_n)
\end{equation}의 확률을 구하는 함수이다.

* calculate_unigram_prob
  * 이 과정에서 로그확률로 바꾸어 구현했다.
\begin{equation}
    P(w_1 w_2 ... w_n) \approx \prod_{i} P(w_i)
\end{equation}
  * for loop을 돌면서 estimate_unigram_prob를 통해 각 P(w_i)를 구하고, 이를 total_prob이라는 것에 log를 씌운 값을 저장해 나간다. 그리고, 마지막에 exponential함수를 이용해서 log scaling한 값을 원상태로 돌려준다.
  * 이 때, 어떤 P(w_i)가 0이 나오면, add one smoothing을 해주어야 한다. 즉, 돌던 for loop을 빠져나와 새로운 for loop으로 진입하는데, 이 때에는 estimate_unigram_prob의 매개변수 중 is_add_one값을 1로 사용하여 add one smoothing된 확률을 내뱉을 수 있도록 한다. code가 올바르게 돌아가는지 체크하기 위해, add one smoothing된 확률이 0이하일 경우, error를 출력하도록 한다.
* calculate_bigram_prob
 * 전반적인 과정은 unigram과 동일하지만, 한 가지 달라진 점은 첫 번째 token을 따로 구해주어야 한다는 것이다. 이를 위해 first_prob이라는 변수를 주어, 이에는 \begin{equation}
    P(w_0)
\end{equation}값이 unigram으로 계산되어 들어가도록 하고, total_prob에 log scaling된 해당 값을  더해주었다. 혹시 모든 i에 대해 \begin{equation}
    P(w_i | w_{i-1})
\end{equation}
  값은 0이 아니지만, P(w_0)는 0인 경우, 전체 과정에 add one smoothing해서 다시 구해저야 하기 때문에, 이를 반영할 수 있도록, 이른 return을 사용하지 않았다.

* calculate_trigram_prob
  * 전반적인 과정은 bigram과 동일하지만, 여기서는 \begin{equation}
    P(w_0)
\end{equation}라는 unigram 확률과 \begin{equation}
    P(w_1 | w_{0})
\end{equation} bigram 확률을 따로 계산해주어야 했다.


In [24]:
import math
def calculate_unigram_prob(token,unigram_count):
  total_prob=0
  is_add_one=0
  for i in range(0,len(token)):
    #print("no_add_one",i)
    prob=estimate_unigram_prob(token[i],unigram_count,0)
    if prob<=0:
      is_add_one=1
      break
    total_prob+=math.log(prob)
  return math.exp(total_prob)

  if(is_add_one==1):
    total_prob=0
    for i in range(0,len(token)):
      #print("yes_add_one",i)
      prob=estimate_unigram_prob(token[i],unigram_count,1)
      if(prob<=0):
        print("ERROR : Add one smoothed probability cannot be 0")
      total_prob+=math.log(prob)
  return math.exp(total_prob)

In [25]:
def calculate_bigram_prob(token,unigram_count,bigram_count):
  total_prob=0
  is_add_one=0
  if is_add_one==0:
    for i in range(1,len(token)):
      #print("no_add_one",i)
      prob=estimate_bigram_prob(token[i-1],token[i],unigram_count,bigram_count,0)
      if prob<=0:
        #print("prob==0")
        is_add_one=1
        break
      total_prob+=math.log(prob)
    first_prob=estimate_unigram_prob(token[0],unigram_count,0)
    if(first_prob==0):
      is_add_one=1

  if(is_add_one==1):
    total_prob=0
    first_prob=estimate_unigram_prob(token[0],unigram_count,1)
    for i in range(1,len(token)):
      #print("yes_add_one",i)
      prob=estimate_bigram_prob(token[i-1],token[i],unigram_count,bigram_count,1)
      if(prob<=0):
        print("ERROR : Add one smoothed probability cannot be 0")
      total_prob+=math.log(prob)
  total_prob+=math.log(first_prob)
  return math.exp(total_prob)

In [26]:
def calculate_trigram_prob(token,unigram_count,bigram_count,trigram_count):
  total_prob=0
  is_add_one=0
  if is_add_one==0:
    for i in range(2,len(token)):
      #print("no_add_one",i)
      prob=estimate_trigram_prob(token[i-2],token[i-1],token[i],unigram_count,bigram_count,trigram_count,0)
      if prob<=0:
        #print("prob==0")
        is_add_one=1
        break
      total_prob+=math.log(prob)
    first_prob=estimate_unigram_prob(token[0],unigram_count,0)
    second_prob=estimate_bigram_prob(token[0],token[1],unigram_count,bigram_count,0)
    if(first_prob==0 or second_prob==0):
      is_add_one=1

  if(is_add_one==1):
    total_prob=0
    first_prob=estimate_unigram_prob(token[0],unigram_count,1)
    second_prob=estimate_bigram_prob(token[0],token[1],unigram_count,bigram_count,1)
    for i in range(1,len(token)):

      prob=estimate_trigram_prob(token[i-2],token[i-1],token[i],unigram_count,bigram_count,trigram_count,1)
      if(prob<=0):
        print("ERROR : Add one smoothed probability cannot be 0")
      total_prob+=math.log(prob)
  total_prob+=math.log(first_prob)+math.log(second_prob)
  return math.exp(total_prob)

In [27]:
print(calculate_unigram_prob(target_token,unigram_count))
print(calculate_bigram_prob(target_token,unigram_count,bigram_count))
print(calculate_trigram_prob(target_token,unigram_count,bigram_count,trigram_count))

8.235480955433522e-56
6.728574202779773e-61
7.589731847614976e-98


# 6.4 get_ngram_prob

편의를 위해 target_sentence와, ngram_count, 그리고 unigram, bigram, trigram 중 어떤 것을 사용할 것인지 n값만 넣어주면, target_sentence가 나타날 ngram 확률을 return해주는 wrapper function을 정의했다.

target_sentence에 대해 word_tokenize function을 이용해 tokenize를 하고, generate_ngrams을 통해 target_ngram을 만든다.

* unigram을 구하고자 하면, target_sentence,1,[unigram_count]를 넣으면, ngram_count_array의 0번째 값을 꺼내어, unigram_count로 정의하고, calculate_unigram_prob함수를 이용해 확률값을 계산해 return한다.
* bigram, trigram도 마찬가지임.


In [28]:
def get_ngram_prob(target_sentence,n,ngram_count_array):
  target_token=word_tokenize(target_sentence,m,punct,punct_mapping)
  if(n==1):
    unigram_count=ngram_count_array[0]
    target_unigram = generate_ngrams(target_token, 1)
    return calculate_unigram_prob(target_token,unigram_count)
  elif(n==2):
    unigram_count=ngram_count_array[0]
    bigram_count=ngram_count_array[1]
    target_unigram = generate_ngrams(target_token, 1)
    target_bigram = generate_ngrams(target_token, 2)
    return calculate_bigram_prob(target_token,unigram_count,bigram_count)
  elif(n==3):
    unigram_count=ngram_count_array[0]
    bigram_count=ngram_count_array[1]
    trigram_count=ngram_count_array[2]
    target_unigram = generate_ngrams(target_token, 1)
    target_bigram = generate_ngrams(target_token, 2)
    target_trigram=generate_ngrams(target_token, 3)
    return calculate_trigram_prob(target_token,unigram_count,bigram_count,trigram_count)

In [29]:
print(get_ngram_prob(target_sentence,1,[unigram_count]))
print(get_ngram_prob(target_sentence,2,[unigram_count,bigram_count]))
print(get_ngram_prob(target_sentence,3,[unigram_count,bigram_count,trigram_count]))

8.235480955433522e-56
6.728574202779773e-61
7.589731847614976e-98


##6.5 make_ngram_and_get_ngram_prob

앞의 function은 만들어진 ngram_count가 있어서 이를 input으로 사용했다.
이번 wrapper function은 End-to-end로, json_path와 target_sentecne, n값만 넣으면, n-gram 확률을 계산해주는 함수로,
지금까지 만들었던 wrapper function들을 넣어서 만들었다.

In [30]:
def make_ngram_and_get_ngram_prob(json_path,target_sentence,n):
  merged_list=make_merged_list(json_path)
  if(n==1):
    unigram_count=generate_ngram_counter(merged_list,1)
    return get_ngram_prob(target_sentence,1,[unigram_count])
  elif(n==2):
    unigram_count=generate_ngram_counter(merged_list,1)
    bigram_count=generate_ngram_counter(merged_list,2)
    return get_ngram_prob(target_sentence,2,[unigram_count,bigram_count])
  elif(n==3):
    unigram_count=generate_ngram_counter(merged_list,1)
    bigram_count=generate_ngram_counter(merged_list,2)
    trigram_count=generate_ngram_counter(merged_list,3)
    return get_ngram_prob(target_sentence,3,[unigram_count,bigram_count,trigram_count])
  else:
    print("ERROR: Please use either unigram, bigram or trigram")


In [31]:
json_path="NLRW1900000011.json"
target_sentence="21세기는 선진 경제와 예술인의 역량을 높이는 일에 앞장서고자 한다"
unigram_prob=make_ngram_and_get_ngram_prob(json_path,target_sentence,1)
print("Unigram Probabilities:")
print(unigram_prob)

Processing  0 / 69731 th paragraph
Processing  10000 / 69731 th paragraph
Processing  20000 / 69731 th paragraph
Processing  30000 / 69731 th paragraph
Processing  40000 / 69731 th paragraph
Processing  50000 / 69731 th paragraph
Processing  60000 / 69731 th paragraph
Unigram Probabilities:
8.235480955433522e-56


# 결과
결과적으로 "21세기는 선진 경제와 예술인의 역량을 높이는 일에 앞장서고자 한다"이 나올 확률을 구한 결과는 아래와 같다.
* unigram : 8.235480955433522e-56
* bigram : 6.728574202779773e-61
* trigram : 7.589731847614976e-98
